In [1]:
import os
import glob
import torch
import random
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [2]:
# Define relevant variables for the ML task
batch_size = 32
num_classes = 2
learning_rate = 0.001
num_epochs = 170
#  Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()
torch.cuda.device_count()

0

In [3]:
data_path = './'
train_file_name = 'train_dataset.csv'
test_file = 'val_dataset.csv'
train_csv_file = os.path.join(data_path,train_file_name)
val_csv_file = os.path.join(data_path,test_file)

In [4]:
class CsvDataset(Dataset):
    def __init__(self, csv_file, desired_action, transform=None):
        self.data = pd.read_csv(csv_file)
        act = desired_action
        val = self.data.iloc[:,1].values
        i=0
        for v in val :
            if v == act:
                self.data.iloc[i,1]=0
            elif v != act :
                self.data.iloc[i,1]=1
            i+=1
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
        label= self.data.iloc[idx,1]
        np_file = self.data.iloc[idx,0]
        np_data = np.load(np_file)
        np_data = torch.from_numpy(np_data).float()
        if self.transform:
            sample = self.transform(np_data)
        return np_data,label

In [5]:
#### Choose desired joint to have label 0 and change all the other labels to value 1
# desired_class = 4
# data = pd.read_csv(train_csv_file)
# val = data.iloc[:,1].values
# print(data.iloc[:,1].values)
# i=0
# for v in val :
#     if v == desired_class:
#         data.iloc[i,1]=0
#     elif v != desired_class :
#         data.iloc[i,1]=1
#     i+=1
# print(data.iloc[:,1].values)


In [6]:
#######################################################
#                  Create Dataset
#######################################################

# Choose desired joint to have label 0 and change all the other labels to value 1
desired_action  = 0

train_dataset = CsvDataset(train_csv_file,desired_action)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

valid_dataset = CsvDataset(val_csv_file,desired_action)
valid_loader = DataLoader(valid_dataset,batch_size, shuffle=False)

dataiter = iter(train_loader)
files, labels = dataiter.next()
# print(files.shape)
# print(labels.shape)

In [7]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=18, kernel_size=(3,13))
        self.batch1 = nn.BatchNorm2d(18)
        self.conv_layer2 = nn.Conv2d(in_channels=18, out_channels=32, kernel_size=(3,13))
        self.batch2 = nn.BatchNorm2d(32)
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,13))
    
        self.sig = nn.Sigmoid()
        self.fc1 = nn.Linear(79872, 164)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(164, num_classes)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.batch1(out)
        out = self.conv_layer2(out)
        out = self.batch2(out)
        out = self.conv_layer3(out)

        out = out.reshape(out.size(0), -1)
#         out = self.relu1(out)
        out = self.sig(out)
        out = self.fc1(out)
        out = self.sig(out)
#         out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
net = Net(num_classes)

if torch.cuda.is_available():
    print("Running at GPU")
    net = net.cuda()
else:
    print("CPU Training")
    

CPU Training


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
total_step = len(train_loader)

In [10]:
min_valid_loss = np.inf

for epoch in range(num_epochs):  # loop over the dataset multiple times
#     last_loss = 0.
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs,labels = data
        if torch.cuda.is_available():
            inputs = inputs.to(device)
            labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
        net.eval()     # Optional when not using Model Specific layer
        with torch.no_grad():
            for data, labels in valid_loader:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target = net(data)
                loss = criterion(target,labels)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {epoch+1} \t\t Training Loss: {running_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                # Saving State Dict
                torch.save(net.state_dict(), 'saved_model_binary.pth')
print('Finished Training')

Epoch [1/170], Loss: 0.6617
Epoch 1 		 Training Loss: 0.33083048462867737 		 Validation Loss: 8.122211515903473
Validation Loss Decreased(inf--->16.244423) 	 Saving The Model
Epoch [1/170], Loss: 0.5998
Epoch 1 		 Training Loss: 0.6307490170001984 		 Validation Loss: 9.289833784103394
Epoch [2/170], Loss: 0.6226
Epoch 2 		 Training Loss: 0.3112880289554596 		 Validation Loss: 10.660493910312653
Epoch [2/170], Loss: 0.4821
Epoch 2 		 Training Loss: 0.5523403733968735 		 Validation Loss: 12.486127316951752
Epoch [3/170], Loss: 0.6086
Epoch 3 		 Training Loss: 0.30427613854408264 		 Validation Loss: 13.923225045204163
Epoch [3/170], Loss: 0.5026
Epoch 3 		 Training Loss: 0.5555747151374817 		 Validation Loss: 15.106056690216064
Epoch [4/170], Loss: 0.6018
Epoch 4 		 Training Loss: 0.30087876319885254 		 Validation Loss: 15.73744922876358
Epoch [4/170], Loss: 0.5819
Epoch 4 		 Training Loss: 0.5918330252170563 		 Validation Loss: 15.89616197347641
Epoch [5/170], Loss: 0.5560
Epoch 5 		 Tra

Epoch [37/170], Loss: 0.4090
Epoch 37 		 Training Loss: 0.20447801053524017 		 Validation Loss: 10.140880465507507
Epoch [37/170], Loss: 0.5626
Epoch 37 		 Training Loss: 0.4857647269964218 		 Validation Loss: 9.962503552436829
Epoch [38/170], Loss: 0.4810
Epoch 38 		 Training Loss: 0.24051213264465332 		 Validation Loss: 9.780445396900177
Epoch [38/170], Loss: 0.4685
Epoch 38 		 Training Loss: 0.4747593253850937 		 Validation Loss: 9.651323407888412
Epoch [39/170], Loss: 0.4162
Epoch 39 		 Training Loss: 0.2080843150615692 		 Validation Loss: 9.73546501994133
Epoch [39/170], Loss: 0.5238
Epoch 39 		 Training Loss: 0.4699784815311432 		 Validation Loss: 9.616464883089066
Epoch [40/170], Loss: 0.4992
Epoch 40 		 Training Loss: 0.24960072338581085 		 Validation Loss: 9.380307108163834
Epoch [40/170], Loss: 0.4186
Epoch 40 		 Training Loss: 0.458891361951828 		 Validation Loss: 9.330162853002548
Epoch [41/170], Loss: 0.4084
Epoch 41 		 Training Loss: 0.2042054533958435 		 Validation Loss:

Epoch [67/170], Loss: 0.3258
Epoch 67 		 Training Loss: 0.16290000081062317 		 Validation Loss: 5.563110262155533
Epoch [67/170], Loss: 0.2623
Epoch 67 		 Training Loss: 0.29406970739364624 		 Validation Loss: 5.530427187681198
Epoch [68/170], Loss: 0.2824
Epoch 68 		 Training Loss: 0.14121916890144348 		 Validation Loss: 5.586835563182831
Epoch [68/170], Loss: 0.3029
Epoch 68 		 Training Loss: 0.292685329914093 		 Validation Loss: 5.456515699625015
Epoch [69/170], Loss: 0.2285
Epoch 69 		 Training Loss: 0.11424342542886734 		 Validation Loss: 5.478401631116867
Epoch [69/170], Loss: 0.3552
Epoch 69 		 Training Loss: 0.2918534353375435 		 Validation Loss: 5.263182878494263
Validation Loss Decreased(10.722448--->10.526366) 	 Saving The Model
Epoch [70/170], Loss: 0.3627
Epoch 70 		 Training Loss: 0.18134936690330505 		 Validation Loss: 5.466608136892319
Epoch [70/170], Loss: 0.2127
Epoch 70 		 Training Loss: 0.28768931329250336 		 Validation Loss: 5.150031924247742
Validation Loss Decrea

Epoch [96/170], Loss: 0.2207
Epoch 96 		 Training Loss: 0.22049156576395035 		 Validation Loss: 2.454783372581005
Validation Loss Decreased(4.940781--->4.909567) 	 Saving The Model
Epoch [97/170], Loss: 0.1338
Epoch 97 		 Training Loss: 0.06691549718379974 		 Validation Loss: 2.3599763587117195
Validation Loss Decreased(4.909567--->4.719953) 	 Saving The Model
Epoch [97/170], Loss: 0.3133
Epoch 97 		 Training Loss: 0.2235768586397171 		 Validation Loss: 2.414818547666073
Epoch [98/170], Loss: 0.1066
Epoch 98 		 Training Loss: 0.05328598618507385 		 Validation Loss: 2.5093991607427597
Epoch [98/170], Loss: 0.3360
Epoch 98 		 Training Loss: 0.2213052362203598 		 Validation Loss: 2.5809943825006485
Epoch [99/170], Loss: 0.2096
Epoch 99 		 Training Loss: 0.1047855094075203 		 Validation Loss: 2.6223054081201553
Epoch [99/170], Loss: 0.2208
Epoch 99 		 Training Loss: 0.21516509354114532 		 Validation Loss: 2.6606877744197845
Epoch [100/170], Loss: 0.2295
Epoch 100 		 Training Loss: 0.114762

Epoch [126/170], Loss: 0.1918
Epoch 126 		 Training Loss: 0.09588846564292908 		 Validation Loss: 1.7270896062254906
Epoch [126/170], Loss: 0.1804
Epoch 126 		 Training Loss: 0.18607021868228912 		 Validation Loss: 1.687702126801014
Epoch [127/170], Loss: 0.1982
Epoch 127 		 Training Loss: 0.09909114986658096 		 Validation Loss: 1.6312781050801277
Validation Loss Decreased(3.364343--->3.262556) 	 Saving The Model
Epoch [127/170], Loss: 0.1716
Epoch 127 		 Training Loss: 0.18487611413002014 		 Validation Loss: 1.600712589919567
Validation Loss Decreased(3.262556--->3.201425) 	 Saving The Model
Epoch [128/170], Loss: 0.1615
Epoch 128 		 Training Loss: 0.08076781779527664 		 Validation Loss: 1.5859189331531525
Validation Loss Decreased(3.201425--->3.171838) 	 Saving The Model
Epoch [128/170], Loss: 0.2086
Epoch 128 		 Training Loss: 0.18506426364183426 		 Validation Loss: 1.5653195902705193
Validation Loss Decreased(3.171838--->3.130639) 	 Saving The Model
Epoch [129/170], Loss: 0.2257
Ep

Epoch [154/170], Loss: 0.2047
Epoch 154 		 Training Loss: 0.102353535592556 		 Validation Loss: 1.2089525386691093
Epoch [154/170], Loss: 0.1128
Epoch 154 		 Training Loss: 0.1587546281516552 		 Validation Loss: 1.2096648328006268
Epoch [155/170], Loss: 0.1478
Epoch 155 		 Training Loss: 0.073881134390831 		 Validation Loss: 1.1549712046980858
Epoch [155/170], Loss: 0.1745
Epoch 155 		 Training Loss: 0.1611405834555626 		 Validation Loss: 1.1584071926772594
Epoch [156/170], Loss: 0.1138
Epoch 156 		 Training Loss: 0.05691374093294144 		 Validation Loss: 1.1865954212844372
Epoch [156/170], Loss: 0.2063
Epoch 156 		 Training Loss: 0.16008342802524567 		 Validation Loss: 1.1775724031031132
Epoch [157/170], Loss: 0.1510
Epoch 157 		 Training Loss: 0.07550046592950821 		 Validation Loss: 1.2066991701722145
Epoch [157/170], Loss: 0.1657
Epoch 157 		 Training Loss: 0.15835725516080856 		 Validation Loss: 1.175278902053833
Epoch [158/170], Loss: 0.1715
Epoch 158 		 Training Loss: 0.08573627471

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in valid_loader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the data: {} %'.format( 100 * correct / total))



Accuracy of the network on the data: 98.11320754716981 %
